In [2]:
#importing dependencies
import pandas as Pd 
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [3]:
csv_path="Resources/fdp-beta-state-historical (1).csv"
csv_file=Pd.read_csv(csv_path,encoding='cp1252')

In [4]:
csv_file


,Variable,Year,Value,RSE,State,Industry
0,AI stud fees and herd testing ($),1990,200.0,118.0,New South Wales,All Broadacre
1,Accounting services ($),1990,2620.0,7.0,New South Wales,All Broadacre
2,Advisory services ($),1990,180.0,105.0,New South Wales,All Broadacre
3,Age of owner manager (yrs),1990,55.0,2.0,New South Wales,All Broadacre
4,Age of spouse (yrs),1990,52.0,2.0,New South Wales,All Broadacre
...,...,...,...,...,...,...
184255,Wheat receipts ($),2022,0.0,NaN,Western Australia,Sheep-Beef
184256,Wheat sold (t),2022,0.0,NaN,Western Australia,Sheep-Beef
184257,Wool cut per head (kg),2022,3.6,5.0,Western Australia,Sheep-Beef
184258,Wool produced (kg),2022,9302.0,16.0,Western Australia,Sheep-Beef


In [28]:
wheat_export=csv_file.loc[(csv_file['Variable']=='Wheat sold (t)'),:]

wheat_export_df=wheat_export[['Variable','Year','Value','State']]

wheat_export_df=wheat_export_df.rename(columns={'Value':'Kt'})
reduced_wheat_export_df=wheat_export_df.reset_index().drop(['index','Variable'],axis=1)
reduced_wheat_export_df

,Year,Kt,State
0,1990,126.0,New South Wales
1,1990,4.0,New South Wales
2,1990,690.0,New South Wales
3,1990,291.0,New South Wales
4,1990,26.0,New South Wales
...,...,...,...
1213,2022,1.0,Western Australia
1214,2022,6789.0,Western Australia
1215,2022,1238.0,Western Australia
1216,2022,99.0,Western Australia


In [30]:
#wheat_export_df.dtypes
avg_wheat_export_by_state=reduced_wheat_export_df.groupby(['State','Year']).mean()
avg_wheat_export_by_state

Kt
State             Year             
New South Wales   1990   195.166667
                  1991   178.833333
                  1992   123.000000
                  1993   204.500000
                  1994   206.500000
...                             ...
Western Australia 2018   800.166667
                  2019  1113.166667
                  2020   710.000000
                  2021  1160.800000
                  2022  1688.666667

[231 rows x 1 columns]